# Swimming Problem

## Import Libraries

In [1]:
import sys
import numpy as np
import pandas as pd

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

## Data Preprocessing

In [2]:
time_values = [[37.7, 32.9, 33.8, 37.0, 35.4],
               [43.4, 33.1, 42.2, 34.7, 41.8],
               [33.3, 28.5, 38.9, 30.4, 33.6],
               [29.2, 26.4, 29.6, 28.5, 31.1]]
stroke_list = ['Backstroke', 'Breaststroke', 'Butterfly', 'Freestyle']
swimmer_list = ['Carl', 'Chris', 'David', 'Tony', 'Ken']

## Construct the Model

In [3]:
def swimming(swimmer_num, stroke_num, time_values):
    # create a model
    model = pyo.ConcreteModel("Swimming Problem")

    # Sets of swimmers and strokes
    model.strokes = pyo.RangeSet(stroke_num)
    model.swimmers = pyo.RangeSet(swimmer_num)

    # decision variables
    model.x = pyo.Var(model.strokes, model.swimmers, domain=pyo.Binary)

    # get times parameters, and i:stroke, j:swimmer
    best_times = {(i, j): time_values[i - 1][j - 1] for i in model.strokes for j in model.swimmers}
    model.Times = pyo.Param(model.strokes, model.swimmers, initialize=best_times)

    # objective function
    @model.Objective(sense=pyo.minimize)
    def swimming_goal(model):
        return sum(model.x[i, j] * model.Times[i, j] for i in model.strokes for j in model.swimmers)

    # constraints
    @model.Constraint(model.swimmers)
    def swimmer_constraint(model, j):
        return sum(model.x[i, j] for i in model.strokes) <= 1

    @model.Constraint(model.strokes)
    def stroke_constraint(model, i):
        return sum(model.x[i, j] for j in model.swimmers) == 1

    return model

## Solve the Model

In [4]:
def solve_model(model):
    SOLVER.solve(model)
    model.display()

In [5]:
model = swimming(5, 4, time_values)
solve_model(model)

Model Swimming Problem

  Variables:
    x : Size=20, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 3) :     0 :   1.0 :     1 : False : False : Binary
        (1, 4) :     0 :   0.0 :     1 : False : False : Binary
        (1, 5) :     0 :  -0.0 :     1 : False : False : Binary
        (2, 1) :     0 :   0.0 :     1 : False : False : Binary
        (2, 2) :     0 :  -0.0 :     1 : False : False : Binary
        (2, 3) :     0 :   0.0 :     1 : False : False : Binary
        (2, 4) :     0 :   1.0 :     1 : False : False : Binary
        (2, 5) :     0 :   0.0 :     1 : False : False : Binary
        (3, 1) :     0 :   0.0 :     1 : False : False : Binary
        (3, 2) :     0 :   1.0 :     1 : False : False : Binary
        (3, 3) :     0 :   0.0 :     1 : False : False : Binary
        (3, 4) :     0 :  -0.0 :    

## Result Visualization

In [6]:
def visualization(model, stroke_list, swimmer_list):
    # show in index
    print("Show in index: ")
    for stroke in model.strokes:
        for swimmer in model.swimmers:
            if model.x[stroke, swimmer].value == 1:
                print(f"The stroke {stroke}: swimmer {swimmer}, the corresponding time is {model.Times[stroke, swimmer]}s. ")
    print()
    print("Show in names: ")
    for stroke in model.strokes:
        for swimmer in model.swimmers:
            if model.x[stroke, swimmer].value == 1:
                print(f"The swimmer for {stroke_list[stroke-1]} is: {swimmer_list[swimmer-1]}, the corresponding time is {model.Times[stroke, swimmer]}s. ")
    print(f"\nThe minimum total time is {pyo.value(model.swimming_goal)}. ")

In [7]:
visualization(model, stroke_list, swimmer_list)

Show in index: 
The stroke 1: swimmer 3, the corresponding time is 33.8s. 
The stroke 2: swimmer 4, the corresponding time is 34.7s. 
The stroke 3: swimmer 2, the corresponding time is 28.5s. 
The stroke 4: swimmer 1, the corresponding time is 29.2s. 

Show in names: 
The swimmer for Backstroke is: David, the corresponding time is 33.8s. 
The swimmer for Breaststroke is: Tony, the corresponding time is 34.7s. 
The swimmer for Butterfly is: Chris, the corresponding time is 28.5s. 
The swimmer for Freestyle is: Carl, the corresponding time is 29.2s. 

The minimum total time is 126.2. 
